# Submission Preparation

In [175]:
import pandas as pd
import os
import math
import numpy as np

SYNTH_DATA_PATH = '../data'
OUT_SYNTH_DATA_PATH = '../output'
SOLUTION_FILE = 'clustering/refineries/solution_20_08_2023_04_33_48.csv'

In [176]:
df_submission = pd.read_csv(os.path.join(SYNTH_DATA_PATH, 'sample_submission.csv'))
df_submission.head()

,year,data_type,source_index,destination_index,value
0,20182019,depot_location,1256,NaN,NaN
1,20182019,depot_location,1595,NaN,NaN
2,20182019,depot_location,1271,NaN,NaN
3,20182019,depot_location,2001,NaN,NaN
4,20182019,depot_location,2201,NaN,NaN


In [177]:
df_submission.dtypes

year                   int64
data_type             object
source_index           int64
destination_index    float64
value                float64
dtype: object

In [178]:
df_submission['data_type'].value_counts()

data_type
biomass_demand_supply    21646
biomass_forecast          4836
pellet_demand_supply       152
depot_location              21
refinery_location            4
Name: count, dtype: int64

In [179]:
df_sol = pd.read_csv(os.path.join(OUT_SYNTH_DATA_PATH, SOLUTION_FILE))
df_sol.columns = ["data_type", "solution"]
df_sol.head()

,data_type,solution
0,b_2018_0_122,7.387925
1,b_2018_1_122,40.431847
2,b_2018_2_122,59.181629
3,b_2018_3_122,74.536720
4,b_2018_4_122,16.531315


In [180]:
df_fc = pd.read_csv(os.path.join(SYNTH_DATA_PATH, 'Biomass_History_Synthetic.csv'))
df_fc.head()

,Index,Latitude,Longitude,2018,2019
0,0,24.66818,71.33144,7.387925,5.180296
1,1,24.66818,71.41106,40.431847,42.126945
2,2,24.66818,71.49069,59.181629,73.203232
3,3,24.66818,71.57031,74.536720,101.067352
4,4,24.66818,71.64994,16.531315,26.086885


In [181]:
df_fc_sol = df_fc.copy()
df_fc_sol = df_fc_sol.melt(value_vars=['2018', '2019'], var_name='year', value_name='biomass', id_vars=['Index'])
df_fc_sol['data_type'] = 'biomass_forecast'
df_fc_sol['destination_index'] = None
df_fc_sol = df_fc_sol[['data_type', 'year', 'Index', 'destination_index', 'biomass']]
df_fc_sol.columns = ['data_type', 'year', 'source_index', 'destination_index', 'value']
df_fc_sol.head()

,data_type,year,source_index,destination_index,value
0,biomass_forecast,2018,0,None,7.387925
1,biomass_forecast,2018,1,None,40.431847
2,biomass_forecast,2018,2,None,59.181629
3,biomass_forecast,2018,3,None,74.536720
4,biomass_forecast,2018,4,None,16.531315


In [182]:
df_fc_sol.loc[(df_fc_sol['data_type'] == 'biomass_forecast')&(df_fc_sol['year'] == '2019'), :]

,data_type,year,source_index,destination_index,value
2418,biomass_forecast,2019,0,None,5.180296
2419,biomass_forecast,2019,1,None,42.126945
2420,biomass_forecast,2019,2,None,73.203232
2421,biomass_forecast,2019,3,None,101.067352
2422,biomass_forecast,2019,4,None,26.086885
...,...,...,...,...,...
4831,biomass_forecast,2019,2413,None,5.321604
4832,biomass_forecast,2019,2414,None,0.120626
4833,biomass_forecast,2019,2415,None,0.038879
4834,biomass_forecast,2019,2416,None,1.304297


## Biomass

In [183]:
df_sol

,data_type,solution
0,b_2018_0_122,7.387925
1,b_2018_1_122,40.431847
2,b_2018_2_122,59.181629
3,b_2018_3_122,74.536720
4,b_2018_4_122,16.531315
...,...,...
4925,x_1984,1.000000
4926,x_1119,1.000000
4927,x_1642,1.000000
4928,x_1757,1.000000


In [184]:
df_sol_proc = df_sol.copy()
df_sol_proc['data_type'] = df_sol_proc['data_type'].str.replace('x_', 'x_20182019_').str.replace('r_', 'r_20182019_')
df_sol_proc = df_sol_proc['data_type'].str.split("_", expand=True)
df_sol_proc.columns = ['data_type', 'year', 'source_index', 'destination_index']

df_sol_proc['value'] = df_sol['solution']
df_sol_proc['data_type'] = df_sol_proc['data_type'].map({'b': 'biomass_demand_supply', 
                                                         'p': 'pellet_demand_supply', 
                                                         'x': 'depot_location', 
                                                         'r': 'refinery_location'})
df_sol_proc = df_sol_proc[df_sol_proc['value'] != 0]

df_sol_proc = pd.concat([df_sol_proc, df_fc_sol])
# df_sol_proc['value'] = df_sol_proc['value'].apply(lambda x: math.ceil(x * 10000) / 10000)
# df_sol_proc.loc[df_sol_proc['data_type'].isin(['biomass_demand_supply']), 'value'] -= 1e-8
# df_sol_proc.loc[df_sol_proc['data_type'].isin(['pellet_demand_supply']), 'value'] -= 1e-8
df_sol_proc.to_csv(os.path.join(OUT_SYNTH_DATA_PATH, 'subm_cluster_and_refs_round.csv'), index=False)
df_sol_proc.head()

,data_type,year,source_index,destination_index,value
0,biomass_demand_supply,2018,0,122,7.387925
1,biomass_demand_supply,2018,1,122,40.431847
2,biomass_demand_supply,2018,2,122,59.181629
3,biomass_demand_supply,2018,3,122,74.536720
4,biomass_demand_supply,2018,4,122,16.531315


In [185]:
df_sol_proc.data_type.value_counts()

data_type
biomass_demand_supply    4858
biomass_forecast         4836
pellet_demand_supply       46
depot_location             22
refinery_location           4
Name: count, dtype: int64

# Solution Checker

## Constraint 2: The processed biomass from each HS <= FC Biomass

In [186]:
forecasted_bio_18[provided_bio_18 <= forecasted_bio_18.iloc[provided_bio_18.index.astype(int), :]].dropna()
(provided_bio_18 - forecasted_bio_18.iloc[provided_bio_18.index.astype(int), :]).values.sum()


-2.4179259532853474e-07

In [187]:
provided_bio_18 = df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'biomass_demand_supply')].\
    groupby('source_index')[['value']].sum()
provided_bio_18.index = provided_bio_18.index.astype(int)
forecasted_bio_18 = df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'biomass_forecast')].\
    groupby('source_index')[['value']].sum().reset_index(drop=True)
# provided_bio_18[provided_bio_18 <= forecasted_bio_18.iloc[provided_bio_18.index.astype(int), :]].dropna()
forecasted_bio_18[provided_bio_18 <= forecasted_bio_18.iloc[provided_bio_18.index.astype(int), :]].dropna()

assert np.all(provided_bio_18.values <= forecasted_bio_18.iloc[provided_bio_18.index.astype(int), :].values), "RESTRICTION 2 NOT SATISFIED 2018: {}"

In [188]:
provided_bio_19 = df_sol_proc[(df_sol_proc['year'] == '2098')&(df_sol_proc['data_type'] == 'biomass_demand_supply')].\
    groupby('source_index')[['value']].sum()

forecasted_bio_19 = df_sol_proc[(df_sol_proc['year'] == '2019')&(df_sol_proc['data_type'] == 'biomass_forecast')].\
    groupby('source_index')[['value']].sum().reset_index(drop=True)

assert np.all(provided_bio_19.values <= forecasted_bio_19.iloc[provided_bio_19.index.astype(int), :].values), "RESTRICTION 2 NOT SATISFIED 2018"

## 3-4. Can't transport more than storage limit

In [189]:
idx_max_stocked_18 = stocked_bio_18.idxmax()[0]
idx_max_stocked_18

'987'

In [190]:
df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'biomass_demand_supply')].\
    groupby('destination_index')[['value']].sum()#.max()

,value
destination_index,
1106,4734.788549
1119,19999.990000
1128,19999.990000
1147,19999.990000
122,6785.854773
1323,1710.215446
1409,2635.464539
1605,19999.990000
1631,2217.513807


In [191]:
df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'pellet_demand_supply')].\
    groupby('source_index')[['value']].sum()#.max()

,value
source_index,
1106,4734.787649
1119,19999.990900
1128,19999.989100
1147,19999.990900
122,6785.853873
1323,1710.214546
1409,2635.463639
1605,19999.989100
1631,2217.512907


In [192]:
stocked_bio_18 = df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'biomass_demand_supply')].\
    groupby('destination_index')[['value']].sum()

stocked_bio_19 = df_sol_proc[(df_sol_proc['year'] == '2019')&(df_sol_proc['data_type'] == 'biomass_demand_supply')].\
    groupby('destination_index')[['value']].sum()

# idx_max_stocked_18 = stocked_bio_18.idxmax()[0]
# df_sol_proc.loc[idx_max_stocked_18, 'value'] = 20000.

max_stocked_18 = stocked_bio_18.max()[0]
max_stocked_19 = stocked_bio_19.max()[0]
print(f"Max stocked biomass in 2018: {max_stocked_18}")
print(f"Max stocked biomass in 2019: {max_stocked_19}")
assert (stocked_bio_18 <= 20000.).values.all(), f"Constraint 3 violated for 2018: {max_stocked_18}"
assert (stocked_bio_19 <= 20000.).values.all(), f"Constraint 3 violated for 2019: {max_stocked_19}"

Max stocked biomass in 2018: 19999.990000000023
Max stocked biomass in 2019: 19999.990000000013


In [193]:
processed_bio_18 = df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'pellet_demand_supply')].\
    groupby('destination_index')[['value']].sum()

processed_bio_19 = df_sol_proc[(df_sol_proc['year'] == '2019')&(df_sol_proc['data_type'] == 'pellet_demand_supply')].\
    groupby('destination_index')[['value']].sum()

# idx_max_processed_18 = processed_bio_18.idxmax()[0]
# df_sol_proc.loc[idx_max_processed_18, 'value'] = 20000.

max_processed_18 = processed_bio_18.max()[0]
max_processed_19 = processed_bio_19.max()[0]
print(f"Max processed biomass in 2018: {max_processed_18}")
print(f"Max processed biomass in 2019: {max_processed_19}")
assert (processed_bio_18 <= 100000.).values.all(), f"Constraint 3 violated for 2018: {max_processed_18}"
assert (processed_bio_19 <= 100000.).values.all(), f"Constraint 3 violated for 2019: {max_processed_18}"

Max processed biomass in 2018: 95181.871312435
Max processed biomass in 2019: 99999.99


In [1445]:
# bm_cond_18 = (df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'biomass_demand_supply')
# if max_stocked_18 > 20000:
#     # increment = 20000. - max_stocked_18
#     df_sol_proc.loc[bm_cond, 'value'] = df_sol_proc.loc[bm_cond, 'value'] - 1e-6

# bm_cond_19 = (df_sol_proc['year'] == '2019')&(df_sol_proc['data_type'] == 'biomass_demand_supply')
# if max_stocked_19 > 20000:
#     increment = 20000. - max_stocked_19
#     df_sol_proc.loc[bm_cond_19, 'value'] = df_sol_proc.loc[bm_cond_19, 'value'] - 1e-6


# assert (df_sol_proc.loc[bm_cond, 'value'] <= 20000.).values.all(), f"Constraint 3 violated for 2018: {max_stocked_18}"
# assert (df_sol_proc.loc[bm_cond, 'value'] <= 20000.).values.all(), f"Constraint 3 violated for 2019: {max_stocked_19}"


In [136]:
pm_cond_18 = (df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'pellet_demand_supply')
pm_cond_19 = (df_sol_proc['year'] == '2019')&(df_sol_proc['data_type'] == 'pellet_demand_supply')

proc_pellet_18 = df_sol_proc[pm_cond_18].\
    groupby('source_index')[['value']].sum()

proc_pellet_19 = df_sol_proc[pm_cond_19].\
    groupby('source_index')[['value']].sum()

max_processed_18 = proc_pellet_18.max()[0]
max_processed_19 = proc_pellet_19.max()[0]


if max_processed_18 > 20000:
    # increment = 20000. - max_stocked_18
    df_sol_proc.loc[pm_cond_18, 'value'] = df_sol_proc.loc[pm_cond_18, 'value'] - 1e-2

if max_processed_19 > 20000:
    # increment = 20000. - max_stocked_18
    df_sol_proc.loc[pm_cond_19, 'value'] = df_sol_proc.loc[pm_cond_19, 'value'] - 1e-2

proc_pellet_18 = df_sol_proc[pm_cond_18].\
    groupby('source_index')[['value']].sum()

proc_pellet_19 = df_sol_proc[pm_cond_19].\
    groupby('source_index')[['value']].sum()

max_processed_18 = proc_pellet_18.max()[0]
max_processed_19 = proc_pellet_19.max()[0]

assert (proc_pellet_18 <= 20000).values.all(), f"Constraint 3 violated for 2018: {max_processed_18}"
assert (proc_pellet_19 <= 20000).values.all(), f"Constraint 3 violated for 2019: {max_processed_19}"

## Constraint 8: Pellets in = Pellets out

In [137]:
pellets_in_18 = stocked_bio_18.copy()
pellets_out_18 = df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'pellet_demand_supply')].\
    groupby('source_index')[['value']].sum()

max_dif = (pellets_out_18 - pellets_in_18).abs().max().values[0]
print(f'Pellets in and out of 2018 do not match: {max_dif}')
assert max_dif <= .001, f'Pellets in and out of 2018 do not match: {max_dif}'


Pellets in and out of 2018 do not match: 0.0009000000063679181


## 7. 80%

In [194]:
processed_18 = df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'pellet_demand_supply')]['value'].sum()
forecast_18 = df_sol_proc[(df_sol_proc['year'] == '2018')&(df_sol_proc['data_type'] == 'biomass_forecast')]['value'].sum()

print('Processed 2018: ', processed_18)
print('Forecast 2018: ', forecast_18)
print("Ratio: ", processed_18/forecast_18)

Processed 2018:  273449.97187025694
Forecast 2018:  273449.981870257
Ratio:  0.9999999634302406


In [195]:
processed_19 = df_sol_proc[(df_sol_proc['year'] == '2019')&(df_sol_proc['data_type'] == 'pellet_demand_supply')]['value'].sum()
forecast_19 = df_sol_proc[(df_sol_proc['year'] == '2019')&(df_sol_proc['data_type'] == 'biomass_forecast')]['value'].sum()

print('Processed 2019: ', processed_19)
print('Forecast 2019: ', forecast_19)
print("Ratio: ", processed_19/forecast_19)

Processed 2019:  384857.011076038
Forecast 2019:  384857.021076038
Ratio:  0.9999999740163243


In [21]:
print("N Depots: ", len(df_sol_proc[df_sol_proc['data_type'] == 'depot_location']))
df_sol_proc[df_sol_proc['data_type'] == 'depot_location']#.head()

N Depots:  22


,data_type,year,source_index,destination_index,value
3991,depot_location,20182019,1886,None,1.0
3992,depot_location,20182019,564,None,1.0
3993,depot_location,20182019,305,None,1.0
3994,depot_location,20182019,2144,None,1.0
3995,depot_location,20182019,1631,None,1.0
3996,depot_location,20182019,1852,None,1.0
3997,depot_location,20182019,1128,None,1.0
3998,depot_location,20182019,1106,None,1.0
3999,depot_location,20182019,1409,None,1.0
4000,depot_location,20182019,2081,None,1.0


In [37]:
print("N Refineries: ", len(df_sol_proc[df_sol_proc['data_type'] == 'refinery_location']))

df_sol_proc[df_sol_proc['data_type'] == 'refinery_location']#.head()

N Refineries:  4


,data_type,year,source_index,destination_index,value
4903,refinery_location,20182019,305,None,1.0
4904,refinery_location,20182019,921,None,1.0
4905,refinery_location,20182019,1605,None,1.0
4906,refinery_location,20182019,1642,None,1.0


In [38]:
df_sol_proc.to_csv(os.path.join(OUT_SYNTH_DATA_PATH, 'subm_cluster_and_refs_round.csv'), index=False)
